In [2]:
!pip install tensorflow-hub

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------ --------- 1.3/1.7 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 8.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
"""
input image size as 224×224 pixels
MobileNetV2 model from TensorFlow Hub expects input images of this shape

creates a sequential model in Keras, where layers are stacked one after another

adds a pre-trained MobileNetV2 model as a Keras layer.
The model URL points to the TensorFlow Hub version of MobileNetV2, a lightweight CNN used for image classification.
input_shape=IMAGE_SHAPE+(3,) sets the input shape to (224, 224, 3):
224×224 is the image size.
3 represents RGB channels (color images).

The MobileNetV2 model is pre-trained on ImageNet, a large dataset with 1,000 object classes.
Given an image of size (224, 224, 3), the model predicts which of the 1,000 categories the image belongs to.
"""
import tensorflow as tf
import tensorflow_hub as hub

IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", 
                   input_shape=IMAGE_SHAPE+(3,))
])

In [ ]:
"""
We are opening a goldfish image and resize it to our preset size
"""
gold_fish = Image.open("goldfish.jpg").resize(IMAGE_SHAPE)
gold_fish

In [ ]:
gold_fish = np.array(gold_fish)/255.0
gold_fish.shape
#(224,224,3)

In [ ]:
"""
This turns the goldfish images from 3d to 4d (224,224,3) to (1,224,224,3)
The new dimenion id batch size since deep learning expects multiple images we add batch size,
Here there is only 1 image in this batch, if there are 100 it will be 100
"""
gold_fish[np.newaxis, ...]

In [ ]:
"""
The pretrained model is stored in classifier
so we pass this goldfish image to the classifer to know model
And the result is a array of probability of each class for this image so it is 1*1000 shape
From these arguments we select the amx arg and get its index
"""
result = classifier.predict(gold_fish[np.newaxis, ...])
result.shape

In [ ]:
predicted_label_index = np.argmax(result)
predicted_label_index

In [ ]:
# tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
"""
The above Downloads the ImageNet class labels (1,000 categories) from TensorFlow's official storage and stores it in ImageNetLabels.txt file

We open and read that file 

We get the first 5 indexes with highest probabilities and  later we get the index with highest probability
"""
image_labels = []
with open("ImageNetLabels.txt", "r") as f:
    image_labels = f.read().splitlines()
image_labels[:5]

In [ ]:
image_labels[predicted_label_index]

In [ ]:
#Load flowers dataset

"""
pathlib gives an windows path object 
"""

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory in which there will be a flower_photos folder
# untar true will unzip it

In [ ]:
data_dir

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

In [ ]:
list(data_dir.glob('*/*.jpg'))[:5]

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
roses = list(data_dir.glob('roses/*'))
roses[:5]

In [ ]:
PIL.Image.open(str(roses[1]))

In [ ]:
tulips = list(data_dir.glob('tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
# Read flowers images from disk into numpy array using opencv
"""
flowers_images_dict stores image file paths for different flower categories.
It uses data_dir.glob('category/*') to get all images in each folder.
"""
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

"""
This dictionary assigns a numeric label (integer) to each flower category. 
It is used for classification tasks where models require numerical labels instead of text
"""
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}


In [ ]:
flowers_images_dict['roses'][:5]

In [ ]:
str(flowers_images_dict['roses'][0])

In [ ]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))

In [ ]:
img.shape

In [ ]:
cv2.resize(img,(224,224)).shape

In [ ]:
"""
X (Features/Input Data) → A list to store image arrays.
y (Labels/Target Data) → A list to store corresponding numeric labels from flowers_labels_dict.

Flower name is the subfolder, for that subfolder and all its images then iterat ethrough images
"""
X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
#Train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
#Preprocessing: scale images

X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

#Make prediction using pre-trained model on new flowers dataset
X[0].shape

In [ ]:
IMAGE_SHAPE+(3,)

"""
IMAGE SHAPE IS 224/224 so by doing this we do 224/224/3
"""

x0_resized = cv2.resize(X[0], IMAGE_SHAPE)
x1_resized = cv2.resize(X[1], IMAGE_SHAPE)
x2_resized = cv2.resize(X[2], IMAGE_SHAPE)

In [ ]:
plt.axis('off')
plt.imshow(X[0])

In [ ]:
plt.axis('off')
plt.imshow(X[1])

In [ ]:
plt.axis('off')
plt.imshow(X[2])

In [ ]:
predicted = classifier.predict(np.array([x0_resized, x1_resized, x2_resized]))
predicted = np.argmax(predicted, axis=1)
predicted

#The variable predicted gives the class name from the pretrained 1000 classes here it predicts shower curatin

"""
image_labels[795] gives shower curtain
"""

In [ ]:
#Now take pre-trained model and retrain it using flowers images
"""
Load a pretrained MobileNetV2 model from TensorFlow Hub, removes its final layer, 
and freeze the rest layers  without changing weights
so it can be used as a feature extractor.
You can then add your own classification head to train a custom model.

Means we add our own final layer

In the pretrained model, let's say we have 5 layers (0,1,2,3,4):

Layers 0 to 3 → Frozen (weights don’t change).
Layer 4 → Removed (it was trained for another task, like ImageNet classification).
New Layer 4 → Added (custom classification layer for your dataset).

Loads the MobileNetV2 model as a Keras layer.
It acts as a feature extractor (removing the last classification layer).
input_shape=(224, 224, 3)

Specifies that the input images must be 224×224 pixels with 3 color channels (RGB).
This is the standard size for MobileNetV2.
trainable=False

Freezes the model's weights (no updates during training).
This means that the pretrained features will be used without modification.
This helps prevent overfitting when training on a small dataset.
"""

feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)



In [ ]:

num_of_flowers = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

In [ ]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=5)

In [ ]:
"""
We can see that this has better accuarcy as this model is based on a model trained on a million images
"""
model.evaluate(X_test_scaled,y_test)